In [1]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import numpy as np
import re

In [8]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Tabelas finais')


In [9]:
df = pd.read_csv('SIAPE_ativos_social.csv')
df

,ano,ministerios,servidores_ativos_ministerios,total_servidores_ativos,perc_servidores
0,1999,"ministerio da ciencia, tecnologia, inovacao e ...",8144,482854,1.69
1,1999,ministerio da cultura,2718,482854,0.56
2,1999,ministerio da educacao,154930,482854,32.09
3,1999,ministerio da previdencia social,1344,482854,0.28
4,1999,ministerio da saude,112256,482854,23.25
...,...,...,...,...,...
134,2018,ministerio da educacao,301391,611916,49.25
135,2018,ministerio da saude,101414,611916,16.57
136,2018,ministerio do desenvolvimento social,34242,611916,5.60
137,2018,ministerio dos direitos humanos,409,611916,0.07


In [11]:
df['ministerios'] = df['ministerios'].str.title() #padronizando

In [14]:
def transformar(nome):
    nome = re.sub(r"\bMinisterio\b", "Ministério",nome)
    nome = re.sub(r"\bCiencia\b", "Ciência",nome)
    nome = re.sub(r"\bInovacao\b", "Inovação",nome)
    nome = re.sub(r"\bComunicacoes\b", "Comunicações",nome)
    nome = re.sub(r"\bEducacao\b", "Educação",nome)
    nome = re.sub(r"\bPrevidencia\b", "Previdência",nome)
    nome = re.sub(r"\bSaude\b", "Saúde",nome)

    return nome

In [15]:
df['ministerios'] = df['ministerios'].apply(transformar)

In [16]:
df['ministerios'].unique()


array(['Ministério Da Ciência, Tecnologia, Inovação E Comunicações',
       'Ministério Da Cultura', 'Ministério Da Educação',
       'Ministério Da Previdência Social', 'Ministério Da Saúde',
       'Ministério Do Desenvolvimento Social',
       'Ministério Do Trabalho E Emprego',
       'Ministério Dos Direitos Humanos'], dtype=object)

In [17]:
df = df.drop(['total_servidores_ativos'], axis=1)

In [18]:
df

,ano,ministerios,servidores_ativos_ministerios,perc_servidores
0,1999,"Ministério Da Ciência, Tecnologia, Inovação E ...",8144,1.69
1,1999,Ministério Da Cultura,2718,0.56
2,1999,Ministério Da Educação,154930,32.09
3,1999,Ministério Da Previdência Social,1344,0.28
4,1999,Ministério Da Saúde,112256,23.25
...,...,...,...,...
134,2018,Ministério Da Educação,301391,49.25
135,2018,Ministério Da Saúde,101414,16.57
136,2018,Ministério Do Desenvolvimento Social,34242,5.60
137,2018,Ministério Dos Direitos Humanos,409,0.07


In [20]:
df.rename(columns={'perc_servidores': 'per_servidores'}, inplace=True)
df

,ano,ministerios,servidores_ativos_ministerios,per_servidores
0,1999,"Ministério Da Ciência, Tecnologia, Inovação E ...",8144,1.69
1,1999,Ministério Da Cultura,2718,0.56
2,1999,Ministério Da Educação,154930,32.09
3,1999,Ministério Da Previdência Social,1344,0.28
4,1999,Ministério Da Saúde,112256,23.25
...,...,...,...,...
134,2018,Ministério Da Educação,301391,49.25
135,2018,Ministério Da Saúde,101414,16.57
136,2018,Ministério Do Desenvolvimento Social,34242,5.60
137,2018,Ministério Dos Direitos Humanos,409,0.07


In [21]:
df['per_servidores'] = df['per_servidores']/100

In [22]:
df

,ano,ministerios,servidores_ativos_ministerios,per_servidores
0,1999,"Ministério Da Ciência, Tecnologia, Inovação E ...",8144,0.0169
1,1999,Ministério Da Cultura,2718,0.0056
2,1999,Ministério Da Educação,154930,0.3209
3,1999,Ministério Da Previdência Social,1344,0.0028
4,1999,Ministério Da Saúde,112256,0.2325
...,...,...,...,...
134,2018,Ministério Da Educação,301391,0.4925
135,2018,Ministério Da Saúde,101414,0.1657
136,2018,Ministério Do Desenvolvimento Social,34242,0.0560
137,2018,Ministério Dos Direitos Humanos,409,0.0007


Subindo para o GBQ


In [23]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

c:\Users\ana.sales_republica\Anaconda\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [24]:
schema= [bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
          bigquery.SchemaField('ministerios','STRING',description='Ministérios observados'),
          bigquery.SchemaField('servidores_ativos_ministerios', 'INTEGER',description='Número de servidores ativos por ministério'),
          bigquery.SchemaField('per_servidores','FLOAT', description='Percentual de servidores ativos por ministério ')]

In [25]:
table_ref = dataset_ref.table('SIAPE_ativos_social') 
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=62fade03-d5df-42a5-896c-6182dbd0ff5c>